## Disponível online

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/carlos-adir/UnB-Courses/blob/main/mntf/trabalho3.ipynb)

Esse python notebook está disponível online no GitHub através do link:

* [GitHub/carlos-adir/UnB-Courses/mntf/trabalho3](https://github.com/carlos-adir/UnB-Courses/blob/main/mntf/trabalho3.ipynb)

In [ ]:
try:
    import numpy as np
    from tqdm import tqdm
    from matplotlib import pyplot as plt
except ModuleNotFoundError:
    import os
    os.system("pip install numpy")
    os.system("pip install tqdm")
    os.system("pip install matplotlib")
    import numpy as np
    import tqdm
    from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from typing import List, Tuple, Callable, Optional
import warnings
warnings.filterwarnings("error")

In [ ]:
def plot_field(xmesh: Tuple[float], ymesh: Tuple[float], zvals: np.ndarray, ax = None):
    xmesh = np.array(xmesh, dtype="float64")
    ymesh = np.array(ymesh, dtype="float64")
    zvals = np.array(zvals, dtype="float64")
    assert xmesh.ndim == 1
    assert ymesh.ndim == 1
    assert zvals.ndim == 2
    assert zvals.shape == (len(ymesh), len(xmesh))
    x, y = np.meshgrid(xmesh, ymesh)
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 10))
    else:
        fig = plt.gcf()
    dx = 0.05*(xmesh[-1]-xmesh[0])/2.
    dy = 0.05*(ymesh[-1]-ymesh[0])/2.
    extent = [xmesh[0]-dx, xmesh[-1]+dx, ymesh[0]-dy, ymesh[-1]+dy]
    im = ax.imshow(zvals[::-1], cmap="viridis", interpolation='nearest', aspect='auto', extent=extent)
    cp = ax.contour(x, y, zvals, 10, colors="k")
    div  = make_axes_locatable(ax)
    cax  = div.append_axes('bottom', size='5%', pad=0.6)
    cbar = plt.colorbar(im, cax=cax, orientation='horizontal')
    return ax
    

# A formulação de Diferenças Finitas

Primeiramente fazemos a formulação de diferenças finitas

$$
\dfrac{\partial T}{\partial t} \approx
\begin{bmatrix}
\dfrac{-1}{2\delta t} & 0 & \dfrac{1}{2\delta t}
\end{bmatrix} \cdot
\begin{bmatrix}
T(x, \ t-\delta t) \\ T(x, \ t) \\ T(x, \ t+\delta t)
\end{bmatrix} $$
$$
\dfrac{\partial^2T}{\partial x} \approx
\begin{bmatrix}
\dfrac{1}{\delta x^2} & \dfrac{-2}{\delta x^2} & \dfrac{1}{\delta x^2}
\end{bmatrix} \cdot
\begin{bmatrix}
T(x-\delta x, \ t) \\ T(x, \ t) \\ T(x+\delta x, \ t)
\end{bmatrix} $$

Então

$$
\left[\dfrac{\partial T}{\partial t}  - \dfrac{\partial^2T}{\partial x}\right]_{\begin{matrix}x=x_i \\ t=t_j\end{matrix}} \approx 
\underbrace{\begin{bmatrix}
 \dfrac{-1}{\delta x^2}  & \dfrac{-1}{\delta t} & \dfrac{2}{\delta x^2} & \dfrac{1}{\delta t} & \dfrac{-1}{\delta x^2}
\end{bmatrix}}_{\text{stencil}}
\begin{bmatrix}
T_{j, i-1} \\ T_{j-1, i} \\ T_{j, i} \\ T_{j+1, i} \\  T_{j, i+1}
\end{bmatrix} $$

Usamos a notação

$$
\begin{cases}T_{j, i} = T(x_i, \ t_j)\\ T_{j, i\pm k} = T(x_i \pm k\cdot \delta x, \ t_j) \\ T_{j \pm k, i} = T(x_i, \ t_j \pm k\cdot \delta t)\end{cases} $$

Tem muitos *stencils* diferentes que podem ser usados. Neste exemplo obtemos um erro de ordem $\Theta(\delta x^2, \ \delta t^2)$, mas tem outros:

* Diferença progressiva de $t$ (2 pontos) e centrada em $x$ (3 pontos)

> \begin{align*}
\left[\dfrac{\partial T}{\partial t}\right]_{\begin{matrix}x=x_i \\ t=t_j\end{matrix}} & = \dfrac{- T_{j,i} + T_{j+1, i}}{\delta t} + \Theta (\delta t) \\
\left[\dfrac{\partial^2 T}{\partial x^2}\right]_{\begin{matrix}x=x_i \\ t=t_j\end{matrix}} & = \dfrac{T_{j,i-1}-2T_{j, i} + T_{j,i+1}}{\delta x^2} + \Theta(\delta x^2)\end{align*}

* Diferença centrada em $t$ (3 pontos) e centrada em $x$ (3 pontos)

> \begin{align*}
\left[\dfrac{\partial T}{\partial t}\right]_{\begin{matrix}x=x_i \\ t=t_j\end{matrix}} & = \dfrac{- T_{j-1,i} + T_{j+1, i}}{2\delta t} + \Theta (\delta t^2) \\
\left[\dfrac{\partial^2 T}{\partial x^2}\right]_{\begin{matrix}x=x_i \\ t=t_j\end{matrix}} & = \dfrac{T_{j,i-1}-2T_{j, i} + T_{j,i+1}}{\delta x^2} + \Theta(\delta x^2)\end{align*}

* Diferença centrada em $t$ (5 pontos) e centrada em $x$ (5 pontos)

> \begin{align*}
\left[\dfrac{\partial T}{\partial t}\right]_{\begin{matrix}x=x_i \\ t=t_j\end{matrix}} & = \dfrac{T_{j-2,i}-8T_{j-1, i} + 8 T_{j+1, i}-T_{j+2,i}}{12\delta t} + \Theta (\delta t^4) \\
\left[\dfrac{\partial^2 T}{\partial x^2}\right]_{\begin{matrix}x=x_i \\ t=t_j\end{matrix}} & = \dfrac{-T_{j,i-2}+16T_{j,i-1}-30T_{j,i}+16T_{j,i+1}-T_{j,i+2}}{12\delta x^2} + \Theta(\delta x^4)\end{align*}

No caso, usaremos apenas a primeira. Para menor erro pode-se utilizar as outras formulações.

### Exercício 1

$$
\dfrac{\partial T}{\partial t} = \dfrac{\partial^2 T}{\partial x^2} \ \ \ \ \ \text{on} \ \Omega
$$

No domínio 

$$\Omega = \underbrace{\left[0, \ 2\right]}_{x} \times \underbrace{\left[0, \ \infty\right)}_{t}$$

E condições de contorno

$$
T(0, \ t) = T(2, \ t) = 0 \ \ \ \ \ \forall 0 \le t
$$

$$
T(x, \ 0) = \sin \left(\frac{1}{2}\pi x\right) \ \ \ \ 0 \le x \le 2
$$

A solução exata é dada por

$$
T_{exact}(x, \ t) = \exp \left(-\frac{1}{4}\pi^2 t\right) \cdot \sin \left(\frac{1}{2}\pi x\right)
$$

Então encontramos a solução numérica

Usando diferenças progressivas em $t$ e diferenças centradas em $x$ com 3 pontos, temos 

$$
\dfrac{- T_{j,i} + T_{j+1, i}}{\delta t} = \dfrac{T_{j,i-1}-2T_{j, i} + T_{j,i+1}}{\delta x^2} 
$$

Isolando $T_{j+1, i}$

$$
T_{j+1, \ i} = T_{j, \ i} + \dfrac{\delta t}{\delta x^2} \left(T_{j,i-1}-2T_{j, i} + T_{j,i+1}\right)
$$ 

Então dividimos a malha: No domínio $n_x$ em $n_{x}$ pontos, e no dominio $t$ em $n_{t}$ pontos.

In [ ]:
xmin, xmax = 0, 2
tmin, tmax = 0, 3
nx, nt = 11, 126
dx = (xmax-xmin)/(nx-1)
dt = (tmax-tmin)/(nt-1)
xmesh = np.linspace(xmin, xmax, nx)
tmesh = np.linspace(tmin, tmax, nt)

Então colocamos os valores $T_{j,i}$ conhecidos no contorno:

$$
T(0, \ t) = 0 \Rightarrow T_{j,0} = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ j = 0, \ 1, \ \cdots, \ n_t-1
$$
$$
T(2, \ t) = 0 \Rightarrow T_{j,n_x-1} = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ j = 0, \ 1, \ \cdots, \ n_t -1
$$
$$
T(x, \ 0) = \sin \dfrac{1}{2}\pi x \Rightarrow T_{0,i} = \sin \dfrac{1}{2}\pi x_i \ \ \ \ \ \ \ \ \ \ i = 0, \ 1, \ \cdots, \ n_x - 1
$$

In [ ]:
T = np.zeros((nt, nx), dtype="float64")  # Result matrix
for j, tj in enumerate(tmesh):
    T[j, 0] = 0
    T[j, nx-1] = 0
for i, xi in enumerate(xmesh):
    T[0, i] = np.sin(0.5*np.pi*xi)

Então fazemos as iterações para encontrar os demais valores

In [ ]:
stencilx = np.array([1, -2, 1]) * dt/dx**2
for j in range(nt-1):
    for i in range(1, nx-1):
        T[j+1, i] = T[j, i] + stencilx @ T[j, i-1:i+2]

In [ ]:
Tanalitic = np.tensordot(np.exp(-np.pi**2 * tmesh/4), np.sin(np.pi*xmesh/2), axes=0)

E mostramos os resultados

In [ ]:
ncurves = 5
indexs = [int(curveindex) for curveindex in np.linspace(0, nt-1, ncurves)]
plt.figure(figsize=(10, 4))
for j in indexs:
    pl = plt.plot(xmesh, Tanalitic[j, :], ls="dotted")
    color = pl[0].get_color()
    plt.plot(xmesh, T[j, :], label=f"t = {tmesh[j]:.2}", color=color)
plt.legend()

In [ ]:
ax = plot_field(xmesh, tmesh, T)
[ax.axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
ax.set_title(r"Numerical solution $T_{num}$ at final time")

In [ ]:
ax = plot_field(xmesh, tmesh, np.abs(T-Tanalitic))
[ax.axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
ax.set_title(r"Error of numerical solution  $|T_{num}-T_{exac}|$")

In [ ]:
del xmesh
del tmesh
del T
del Tanalitic

### Exercicio 2


$$
\dfrac{\partial T}{\partial t} = \dfrac{\partial^2 T}{\partial x^2} \ \ \ \ \ \text{on} \ \Omega
$$

No dominio 

$$\Omega = \underbrace{\left[0, \ 1\right]}_{x} \times \underbrace{\left[0, \ \infty\right)}_{t}$$

E condições de contorno

$$
T(0, \ t) = T(1, \ t) = 0 \ \ \ \ \ \forall 0 \le t
$$

$$
T(x, \ 0) = 1 \ \ \ \ 0 < x < 1
$$

A solução exata é dada por

$$
T_{exact}(x, \ t) = \sum_{n=1}^{\infty} \dfrac{4}{(2n-1)\pi} \cdot \exp \left(-(2n-1)^2\pi^2 t\right) \cdot \sin \left((2n-1)\pi x\right)
$$

Então resolvemos numericamente

In [ ]:
xmin, xmax = 0, 1
tmin, tmax = 0, 0.3
nx, nt = 51, 1497
dx = (xmax-xmin)/(nx-1)
dt = (tmax-tmin)/(nt-1)
xmesh = np.linspace(xmin, xmax, nx)
tmesh = np.linspace(tmin, tmax, nt)

T = np.zeros((nt, nx), dtype="float64")  # Result matrix
for i, xi in enumerate(xmesh):
    T[0, i] = 1
for j, tj in enumerate(tmesh):
    T[j, 0] = 0
    T[j, nx-1] = 0

stencilx = np.array([1, -2, 1]) * dt/dx**2
for j in range(nt-1):
    for i in range(1, nx-1):
        T[j+1, i] = T[j, i] + stencilx @ T[j, i-1:i+2]

In [ ]:
Tanalitic = np.zeros(T.shape)
for n in range(1, 21):
    exppart = np.exp(-(2*n-1)**2*(np.pi**2)*tmesh)
    sinpart = np.sin((2*n-1)*np.pi*xmesh)
    Tanalitic += np.tensordot(exppart, sinpart, axes=0)/(2*n-1)
Tanalitic *= 4/np.pi

E mostramos os resultados

In [ ]:
ncurves = 5
indexs = [int(curveindex) for curveindex in np.linspace(0, nt-1, ncurves)]
plt.figure(figsize=(10, 4))
for j in indexs:
    pl = plt.plot(xmesh, Tanalitic[j, :], ls="dotted")
    color = pl[0].get_color()
    plt.plot(xmesh, T[j, :], label=f"t = {tmesh[j]:.2}", color=color)
plt.legend()

In [ ]:
ax = plot_field(xmesh, tmesh, T)
[ax.axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
ax.set_title(r"Numerical solution $T_{num}$ at final time")

In [ ]:
ax = plot_field(xmesh, tmesh, np.abs(T-Tanalitic))
[ax.axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
ax.set_title(r"Error of numerical solution  $|T_{num}-T_{exac}|$")

In [ ]:
del xmesh
del tmesh
del T
del Tanalitic

### Exercicio 3


$$
\dfrac{\partial T}{\partial t} = \dfrac{\partial^2 T}{\partial x^2} \ \ \ \ \ \text{on} \ \Omega
$$

No domínio 

$$\Omega = \underbrace{\left[0, \ 1\right]}_{x} \times \underbrace{\left[0, \ \infty\right)}_{t}$$

E condições de contorno

$$
T(0, \ t) = 1 \ \ \ \ \ \forall 0 \le t
$$
$$
T(1, \ t) = 0 \ \ \ \ \ \forall 0 \le t
$$

$$
T(x, \ 0) = 0 \ \ \ \ 0 < x < 1
$$

A solucao exata é dada por

$$
T_{exact}(x, \ t) = 1-x-\sum_{n=1}^{\infty} \dfrac{2}{n\pi} \cdot \exp \left(-n^2\pi^2 t\right) \cdot \sin \left(n\pi x\right)
$$

Então resolvemos numericamente

In [ ]:
xmin, xmax = 0, 1
tmin, tmax = 0, 0.3
nx, nt = 51, 1497
dx = (xmax-xmin)/(nx-1)
dt = (tmax-tmin)/(nt-1)
xmesh = np.linspace(xmin, xmax, nx)
tmesh = np.linspace(tmin, tmax, nt)

T = np.zeros((nt, nx), dtype="float64")  # Result matrix
for i, xi in enumerate(xmesh):
    T[0, i] = 0
for j, tj in enumerate(tmesh):
    T[j, 0] = 1
    T[j, nx-1] = 0

stencilx = np.array([1, -2, 1]) * dt/dx**2
for j in range(nt-1):
    for i in range(1, nx-1):
        T[j+1, i] = T[j, i] + stencilx @ T[j, i-1:i+2]

In [ ]:
Tanalitic = np.zeros(T.shape)
for n in range(1, 21):
    exppart = np.exp(-n**2*(np.pi**2)*tmesh)
    sinpart = np.sin(n*np.pi*xmesh)
    Tanalitic -= np.tensordot(exppart, sinpart, axes=0)/n
Tanalitic *= 2/np.pi
for i, xi in enumerate(xmesh):
    Tanalitic[:, i] += 1-xi

In [ ]:
ncurves = 5
indexs = [int(curveindex) for curveindex in np.linspace(0, nt-1, ncurves)]
plt.figure(figsize=(10, 4))
for j in indexs:
    pl = plt.plot(xmesh, Tanalitic[j, :], ls="dotted")
    color = pl[0].get_color()
    plt.plot(xmesh, T[j, :], label=f"t = {tmesh[j]:.2}", color=color)
plt.legend()

In [ ]:
ax = plot_field(xmesh, tmesh, T)
[ax.axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
ax.set_title(r"Numerical solution $T_{num}$ at final time")

In [ ]:
ax = plot_field(xmesh, tmesh, np.abs(T-Tanalitic))
[ax.axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
ax.set_title(r"Error of numerical solution  $|T_{num}-T_{exac}|$")

In [ ]:
del xmesh
del tmesh
del T
del Tanalitic

### Exercicio 4


$$
\dfrac{\partial T}{\partial t} = 0.01\cdot \dfrac{\partial^2 T}{\partial x^2} \ \ \ \ \ \text{on} \ \Omega
$$

No dominio


$$\Omega = \underbrace{\left[0, \ 1\right]}_{x} \times \underbrace{\left[0, \ \infty\right)}_{t}$$

E condições de contorno

$$
T(0, \ t) = 0 \ \ \ \ \ \forall 0 \le t
$$
$$
T(1, \ t) = 0 \ \ \ \ \ \forall 0 \le t
$$

$$
T(x, \ 0) = \begin{cases} 200x \ \ \  \ \ \  \ \ \  \ \ \ \text{if}  \ \ \ \ 0 \le x \le 0.5 \\
200(1-x) \ \ \text{if} \ \ \ \  0.5 < x \le 1 \end{cases}
$$

A solução analítica é dada por

$$
T_{exact}(x, \ t) = \dfrac{800}{\pi^2}\sum_{n=0}^{\infty} \dfrac{(-1)^{n}}{(2n+1)^2} \cdot \exp \left(-(2n+1)^2\pi^2 \cdot 0.01 \cdot t\right) \cdot \sin \left((2n+1)\pi x\right)
$$

Então resolvemos numericamente

In [ ]:
xmin, xmax = 0, 1
tmin, tmax = 0, 3
nx, nt = 11, 10001
dx = (xmax-xmin)/(nx-1)
dt = (tmax-tmin)/(nt-1)
xmesh = np.linspace(xmin, xmax, nx)
tmesh = np.linspace(tmin, tmax, nt)

T = np.zeros((nt, nx), dtype="float64")  # Result matrix
for i, xi in enumerate(xmesh):
    if xi > 0.5:
        T[0, i] = 200*(1-xi)
    else:
        T[0, i] = 200*xi
for j, tj in enumerate(tmesh):
    T[j, 0] = 0
    T[j, nx-1] = 0

stencilx = 0.01*np.array([1, -2, 1]) * dt/dx**2
for j in range(nt-1):
    for i in range(1, nx-1):
        T[j+1, i] = T[j, i] + stencilx @ T[j, i-1:i+2]

In [ ]:
Tanalitic = np.zeros(T.shape)
for n in range(0, 21):
    exppart = np.exp(-(2*n+1)**2*(np.pi**2)*0.01*tmesh)
    sinpart = np.sin((2*n+1)*np.pi*xmesh)
    Tanalitic += (-1)**n * np.tensordot(exppart, sinpart, axes=0)/((2*n+1)**2)
Tanalitic *= 800/(np.pi**2)

In [ ]:
ncurves = 5
indexs = [int(curveindex) for curveindex in np.linspace(0, nt-1, ncurves)]
plt.figure(figsize=(10, 4))
for j in indexs:
    pl = plt.plot(xmesh, Tanalitic[j, :], ls="dotted")
    color = pl[0].get_color()
    plt.plot(xmesh, T[j, :], label=f"t = {tmesh[j]:.2}", color=color)
plt.legend()

In [ ]:
ax = plot_field(xmesh, tmesh, T)
[ax.axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
ax.set_title(r"Numerical solution $T_{num}$ at final time")

In [ ]:
ax = plot_field(xmesh, tmesh, np.abs(T-Tanalitic))
[ax.axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
ax.set_title(r"Error of numerical solution  $|T_{num}-T_{exac}|$")

In [ ]:
del xmesh
del tmesh
del T
del Tanalitic

### Exercicio 5

> $$\dfrac{\partial T}{\partial t} = 0.01\cdot \dfrac{\partial^2 T}{\partial x^2} \ \ \ \ \ \text{on} \ \Omega$$
>
> No dominio
> 
> $$\Omega = \underbrace{\left[0, \ 0.5\right]}_{x} \times \underbrace{\left[0, \ \infty\right)}_{t}$$
> 
> E condições de contorno
> 
> $$T(0, \ t) = 0 \ \ \ \ \ \forall \ 0 \le t$$
> $$\left[\dfrac{\partial T}{\partial x}\right]_{x=0.5}= 0 \ \ \ \ \ \forall \ 0 \le t$$
> $$T(x, \ 0) = 200x \ \ \ \ \ \forall  \ 0 \le x \le 0.5$$
> 
> A solução analítica é dada por
> 
> $$T_{exact}(x, \ t) = \dfrac{800}{\pi^2}\sum_{n=0}^{\infty} \dfrac{(-1)^{n}}{(2n+1)^2} \cdot \exp \left(-(2n+1)^2\pi^2 \cdot 0.01 \cdot t\right) \cdot \sin \left((2n+1)\pi x\right)$$

Então resolvemos numericamente


Neste caso temos uma diferença do exercício 4 pois este envolve a condição de Neumann (da derivada) e precisamos de uma reformulação. Até então o nosso método fornece um erro $\Theta(\delta t, \ \delta x^2)$ e queremos mantê-lo em $\delta x^2$

$$\left[\dfrac{\partial T}{\partial t} - \dfrac{\partial^2 T}{\partial x^2}\right]_{\begin{smallmatrix}x=x_{i} \\ t=t_k\end{smallmatrix}} = \dfrac{T_{k+1,i} - T_{k,i}}{\delta t} + \dfrac{-T_{k,i-1} + 2T_{k,i}- T_{k,i+1}}{\delta x^2} + \Theta(\delta t, \ \delta x^2) \ \ \ \ \ \ \ (5.1)$$

A derivada em $x$ pode ser descrita na borda $x = x_{n_{x}-1} = 0.5$ por:

$$
\left[\dfrac{\partial T}{\partial x}\right]_{x=x_{n_{x}-1}} = \dfrac{T_{k,n_{x}} - T_{k,n_{x}-2}}{2\delta x} + \Theta(\delta x^2)
$$

Sendo esse valor igual a zero teremos:

$$
\dfrac{T_{k,n_{x}} - T_{k,n_{x}-2}}{2\delta x}  = 0 \Rightarrow T_{k,n_{x}} = T_{k, n_{x}-2} \ \ \ \ \ \ \ (5.2)
$$

Contudo, na implementação numérica o valor $T_{k,n_x}$ não existe, pois adicionamos um nó fantasma em $x=x_{n_x}$.
Essa restrição diz que 'existe' um ponto simétrico além da fronteira, mas essa restrição não é suficiente para implementar, pois temos uma equação para um grau de liberdade (com o nó fantasma) que devemos relacionar, necessitando de outra equação.
Ela vem da aplicação 
Logo, aplicando a Equação $(5.1)$ no nó $x=x_{n_x-1}$ teremos


$$
\dfrac{T_{k+1,n_x-1} - T_{k,n_x-1}}{\delta t} + 0.01 \cdot \dfrac{-T_{k,n_x-2} + 2T_{k,n_x-1}- T_{k,n_x}}{\delta x^2} = 0
$$

que aplicando a Equação $(5.2)$ obteremos $(5.3)$

$$
\dfrac{T_{k+1,n_x-1} - T_{k,n_x-1}}{\delta t} + 0.01 \cdot \dfrac{2(T_{k,n_x-1}-T_{k,n_x-2})}{\delta x^2} = 0
$$
$$
T_{k+1, n_x-1} = \begin{bmatrix}\dfrac{0.02\delta t}{\delta x^2} & 1 -\dfrac{0.02\delta t}{\delta x^2}\end{bmatrix} 
\begin{bmatrix} T_{k,n_x-2} \\ T_{k,n_x-1}\end{bmatrix} \ \ \ \ \ \ \ (5.3)
$$


In [ ]:
xmin, xmax = 0, 0.5
tmin, tmax = 0, 3
nx, nt = 21, 20000
dx = (xmax-xmin)/(nx-1)
dt = (tmax-tmin)/(nt-1)
xmesh = np.linspace(xmin, xmax, nx)
tmesh = np.linspace(tmin, tmax, nt)

T = np.zeros((nt, nx), dtype="float64")  # Result matrix
for j, tj in enumerate(tmesh):
    T[j, 0] = 0
for i, xi in enumerate(xmesh):
    T[0, i] = 200*xi

const = 0.01*dt/(dx**2)
stencilx = const*np.array([1, -2, 1])
for k in range(nt-1):
    T[k+1, nx-1] = 2*const*T[k, nx-2] + (1-2*const)*T[k,nx-1]
    for i in range(1, nx-1):
        T[k+1, i] = T[k, i] + stencilx @ T[k, i-1:i+2]
    

In [ ]:
Tanalitic = np.zeros(T.shape)
for n in range(0, 21):
    exppart = np.exp(-(2*n+1)**2*(np.pi**2)*0.01*tmesh)
    sinpart = np.sin((2*n+1)*np.pi*xmesh)
    Tanalitic += (-1)**n * np.tensordot(exppart, sinpart, axes=0)/((2*n+1)**2)
Tanalitic *= 800/(np.pi**2)

In [ ]:
ncurves = 5
indexs = [int(curveindex) for curveindex in np.linspace(0, nt-1, ncurves)]
plt.figure(figsize=(10, 4))
for j in indexs:
    pl = plt.plot(xmesh, Tanalitic[j, :], ls="dotted")
    color = pl[0].get_color()
    plt.plot(xmesh, T[j, :], label=f"t = {tmesh[j]:.2}", color=color)
plt.legend()

In [ ]:
ax = plot_field(xmesh, tmesh, T)
[ax.axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
ax.set_title(r"Numerical solution $T_{num}$ at final time")

In [ ]:
ax = plot_field(xmesh, tmesh, np.abs(T-Tanalitic))
[ax.axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
ax.set_title(r"Error of numerical solution  $|T_{num}-T_{exac}|$")

In [ ]:
del xmesh
del tmesh
del T
del Tanalitic

### Exercicio 6

$$
\dfrac{\partial T}{\partial t} = \dfrac{\partial^2 T}{\partial x^2}+\dfrac{\partial^2 T}{\partial y^2}
$$

No domínio

$$
\Omega = \left[0,\ 1\right]\times\left[0,\ 1\right]\times\left[0,\ \infty\right)
$$

E condições iniciais

\begin{align*}
T(x, \ 0, \ t) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(x, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right) \\
T(x, \ 1, \ t) & = \sin \left(\pi x\right) \ \ \ \ \ \forall \left(x, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right)  \\
T(0, \ y, \ t) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(y, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right) \\
T(1, \ y, \ t) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(y, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right)\\
T(x, \ y, \ 0) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(x, \ y\right) \in \left[0, \ 1\right]\times\left[0, \ 1\right]
\end{align*}

A solução analítica estacionária quando $t\to \infty$ é dada por

$$
T_{analitic}(x, \ y,\ t\to \infty) = \dfrac{\sinh (\pi y) \sin (\pi x)}{\sinh \pi}
$$

Então resolvemos numericamente

In [ ]:
xmin, xmax = 0, 1
ymin, ymax = 0, 1
tmin, tmax = 0, 3
nx, ny, nt = 10, 11, 1500
dx = (xmax-xmin)/(nx-1)
dy = (ymax-ymin)/(ny-1)
dt = (tmax-tmin)/(nt-1)
xmesh = np.linspace(xmin, xmax, nx)
ymesh = np.linspace(ymin, ymax, ny)
tmesh = np.linspace(tmin, tmax, nt)

T = np.zeros((nt, ny, nx), dtype="float64")  # Result matrix
T[:, ny-1] = np.sin(np.pi*xmesh)

stencilx = np.array([1, -2, 1])*dt/(dx**2)
stencily = np.array([1, -2, 1])*dt/(dy**2)
for k in tqdm(range(nt-1)):
    for i in range(1, nx-1):
        for j in range(1, ny-1):
            T[k+1, j, i] = T[k, j, i] + stencilx @ T[k, j, i-1:i+2] + stencily @ T[k, j-1:j+2, i]

In [ ]:
sinhpiy = np.sinh(np.pi*ymesh)
sinpix = np.sin(np.pi*xmesh)
Tanalitic = np.tensordot(sinhpiy, sinpix, axes=0)/np.sinh(np.pi)

In [ ]:
ax = plot_field(xmesh, ymesh, T[-1])
[ax.axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
[ax.axhline(y=yi, color="k", ls="dotted") for yi in ymesh]
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Space in $y$")
ax.set_title(r"Numerical solution $T_{num}$ at final time")

In [ ]:
ax = plot_field(xmesh, ymesh, np.abs(T[-1]-Tanalitic))
[ax.axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
[ax.axhline(y=yi, color="k", ls="dotted") for yi in ymesh]
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Space in $y$")
ax.set_title(r"Error of numerical solution  $|T_{num}-T_{exac}|$")

In [ ]:
del xmesh
del ymesh
del tmesh
del T
del Tanalitic

### Exercicio 7

$$
\dfrac{\partial T}{\partial t} = \dfrac{\partial^2 T}{\partial x^2}+\dfrac{\partial^2 T}{\partial y^2}
$$

No domínio

$$
\Omega = \left[0,\ 0.5\right]\times\left[0,\ 1\right]\times\left[0,\ \infty\right)
$$

E condições iniciais

\begin{align*}
T(x, \ 0, \ t) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(x, \ t\right) \in \left[0, \ \frac{1}{2}\right]\times\left[0,\ \infty\right) \\
T(x, \ 1, \ t) & = \sin \left(\pi x\right) \ \ \ \ \ \forall \left(x, \ t\right) \in \left[0, \ \frac{1}{2}\right]\times\left[0,\ \infty\right)  \\
T(0, \ y, \ t) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(y, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right) \\
\dfrac{\partial T}{\partial x}\left(\frac{1}{2}, \ y, \ t\right) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(y, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right)\\
T(x, \ y, \ 0) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(x, \ y\right) \in \left[0, \ \frac{1}{2}\right]\times\left[0, \ 1\right]
\end{align*}

A solução analítica estacionária quando $t\to \infty$ é dada por

$$
T_{analitic}(x, \ y,\ t\to \infty) = \dfrac{\sinh (\pi y) \sin (\pi x)}{\sinh \pi}
$$

Então resolvemos numericamente

In [ ]:
xmin, xmax = 0, 0.5
ymin, ymax = 0, 1
tmin, tmax = 0, 3
nx, ny, nt = 31, 31, 30001
dx = (xmax-xmin)/(nx-1)
dy = (ymax-ymin)/(ny-1)
dt = (tmax-tmin)/(nt-1)
xmesh = np.linspace(xmin, xmax, nx)
ymesh = np.linspace(ymin, ymax, ny)
tmesh = np.linspace(tmin, tmax, nt)

T = np.zeros((nt, ny, nx), dtype="float64")  # Result matrix
for k, tk in enumerate(tmesh):
    T[k, ny-1, :] = np.sin(np.pi*xmesh)
stencilx = np.array([1, -2, 1])*dt/(dx**2)
stencily = np.array([1, -2, 1])*dt/(dy**2)
for k in tqdm(range(nt-1)):
    T[k+1, 1:-1, 1:-1] = (1+stencilx[1]+stencily[1]) * T[k, 1:-1, 1:-1]
    T[k+1, 1:-1, 1:-1] += stencily[0] * T[k, 0:-2, 1:-1]
    T[k+1, 1:-1, 1:-1] += stencily[2] * T[k, 2:, 1:-1]
    T[k+1, 1:-1, 1:-1] += stencilx[0] * T[k, 1:-1, 0:-2]
    T[k+1, 1:-1, 1:-1] += stencilx[2] * T[k, 1:-1, 2:]
    T[k+1, :, nx-1] = T[k+1, :, nx-2]

In [ ]:
sinhpiy = np.sinh(np.pi*ymesh)
sinpix = np.sin(np.pi*xmesh)
Tanalitic = np.tensordot(sinhpiy, sinpix, axes=0)/np.sinh(np.pi)

In [ ]:
ax = plot_field(xmesh, ymesh, T[-1])
[ax.axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
[ax.axhline(y=yi, color="k", ls="dotted") for yi in ymesh]
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Space in $y$")
ax.set_title(r"Numerical solution $T_{num}$ at final time")

In [ ]:
ax = plot_field(xmesh, ymesh, np.abs(T[-1]-Tanalitic))
[ax.axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
[ax.axhline(y=yi, color="k", ls="dotted") for yi in ymesh]
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Space in $y$")
ax.set_title(r"Error of numerical solution  $|T_{num}-T_{exac}|$")

In [ ]:
del xmesh
del ymesh
del tmesh
del T
del Tanalitic

### Exercicio 8

$$
\dfrac{\partial T}{\partial t} = \dfrac{\partial^2 T}{\partial x^2}+\dfrac{\partial^2 T}{\partial y^2}
$$

No domínio

$$
\Omega = \underbrace{\left[0, \ 1\right]}_{x} \times \underbrace{\left[0, \ 1\right]}_{y} \times \underbrace{\left[0, \ \infty\right)}_{t}
$$

And boundary conditions

\begin{align*}
T(x, \ 0, \ t) & = 0  \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(x, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right) \\
T(x, \ 1, \ t) & = \begin{cases}75x  \ \ \ \ \  \ \ \ \ \  \ \ \ \ \  \ \ \forall \left(x, \ t\right) \in \left[0, \ \frac{2}{3}\right]\times\left[0,\ \infty\right)  \\150(1-x)  \ \ \ \ \ \forall \left(x, \ t\right) \in \left(\frac{2}{3}, \ 1\right]\times\left[0,\ \infty\right) \end{cases} \\
T(0, \ y, \ t) & = 0  \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(y, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right) \\
T(1, \ y, \ t) & = 0   \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(y, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right)\\
T(x, \ y, \ 0) & = 0   \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(x, \ y\right) \in \left[0, \ 1\right]\times\left[0, \ 1\right]
\end{align*}


A solução analítica estacionária quando $t\to \infty$ é dada por

$$
T_{exact}(x, \ y,\ t\to \infty) = \dfrac{450}{\pi^2} \cdot \sum_{n=1}^{\infty}\dfrac{\sin \frac{2}{3}n\pi}{n^2 \sinh n\pi} \cdot \sinh (n\pi y) \sin (n \pi x)
$$

Então resolvemos numericamente

In [ ]:
xmin, xmax = 0, 1
ymin, ymax = 0, 1
tmin, tmax = 0, 10
nx, ny, nt = 41, 51, 100001
dx = (xmax-xmin)/(nx-1)
dy = (ymax-ymin)/(ny-1)
dt = (tmax-tmin)/(nt-1)
xmesh = np.linspace(xmin, xmax, nx)
ymesh = np.linspace(ymin, ymax, ny)
tmesh = np.linspace(tmin, tmax, nt)

T = np.zeros((nt, ny, nx), dtype="float64")  # Result matrix
for k, tk in enumerate(tmesh):
    T[k, ny-1, :] = 75*xmesh * (xmesh <= 2/3) +  150*(1-xmesh) * (xmesh > 2/3) 

stencilx = np.array([1, -2, 1])*dt/(dx**2)
stencily = np.array([1, -2, 1])*dt/(dy**2)
for k in tqdm(range(nt-1)):
    T[k+1, 1:-1, 1:-1] = (1+stencilx[1]+stencily[1]) * T[k, 1:-1, 1:-1]
    T[k+1, 1:-1, 1:-1] += stencily[0] * T[k, 0:-2, 1:-1]
    T[k+1, 1:-1, 1:-1] += stencily[2] * T[k, 2:, 1:-1]
    T[k+1, 1:-1, 1:-1] += stencilx[0] * T[k, 1:-1, 0:-2]
    T[k+1, 1:-1, 1:-1] += stencilx[2] * T[k, 1:-1, 2:]

In [ ]:
Tanalitic = np.zeros((ny, nx), dtype="float64")
for n in range(1, 41):
    const = np.sin(2*n*np.pi/3) / (n**2 * np.sinh(n*np.pi))
    sinnpix = np.sin(n*np.pi*xmesh)
    sinhnpiy = np.sinh(n*np.pi*ymesh)
    Tanalitic += const * np.tensordot(sinhnpiy, sinnpix, axes=0)
Tanalitic *= 450/np.pi**2

In [ ]:
ax = plot_field(xmesh, ymesh, T[-1])
[ax.axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
[ax.axhline(y=yi, color="k", ls="dotted") for yi in ymesh]
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Space in $y$")
ax.set_title(r"Numerical solution $T_{num}$ at final time")

In [ ]:
ax = plot_field(xmesh, ymesh, np.abs(T[-1]-Tanalitic))
[ax.axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
[ax.axhline(y=yi, color="k", ls="dotted") for yi in ymesh]
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Space in $y$")
ax.set_title(r"Error of numerical solution  $|T_{num}-T_{exac}|$")

In [ ]:
del xmesh
del ymesh
del tmesh
del T
del Tanalitic

### Exercicio 9

$$
\dfrac{\partial T}{\partial t} = \dfrac{\partial^2 T}{\partial x^2}+\dfrac{\partial^2 T}{\partial y^2}
$$

No domínio

\begin{align*}
S & = \left[0, \ 1\right] \times \left[0, \ 1\right] \\
C & = \left[0.4, \ 0.6\right] \times \left[0.4, \ 0.6\right]  \\
\Omega & = \left(S \setminus C \right) \times \left[0, \ \infty\right) \\
\end{align*}

And boundary conditions

\begin{align*}
T(x, \ 0, \ t) & = 0  \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(x, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right) \\
T(x, \ 1, \ t) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(x, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right) \\
T(0, \ y, \ t) & = 0  \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(y, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right) \\
T(1, \ y, \ t) & = 0   \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(y, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right)\\
T(x, \ y, \ 0) & = 0   \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(x, \ y\right) \in  \left(S \setminus C \right) \\
T(x, \ y, \ 0) & = 1   \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(x, \ y\right) \in C
\end{align*}


A solução analítica estacionária quando $t\to \infty$ é dada por

$$
T_{exact}(x, \ y,\ t\to \infty) = \dfrac{450}{\pi^2} \cdot \sum_{n=1}^{\infty}\dfrac{\sin \frac{2}{3}n\pi}{n^2 \sinh n\pi} \cdot \sinh (n\pi y) \sin (n \pi x)
$$

Então resolvemos numericamente

In [ ]:
xmin, xmax = 0, 1
ymin, ymax = 0, 1
tmin, tmax = 0, 3
nx, ny, nt = 31, 31, 15001
dx = (xmax-xmin)/(nx-1)
dy = (ymax-ymin)/(ny-1)
dt = (tmax-tmin)/(nt-1)
xmesh = np.linspace(xmin, xmax, nx)
ymesh = np.linspace(ymin, ymax, ny)
tmesh = np.linspace(tmin, tmax, nt)

T = np.zeros((nt, ny, nx), dtype="float64")  # Result matrix
xmask = (0.4 <= xmesh) * (xmesh <= 0.6)
ymask = (0.4 <= ymesh) * (ymesh <= 0.6)
Cmask = np.tensordot(ymask, xmask, axes=0)
Imask = ~Cmask[1:-1, 1:-1]
for k, tk in enumerate(tmesh):
    T[k, :, :] = Cmask

stencilx = np.array([1, -2, 1])*dt/(dx**2)
stencily = np.array([1, -2, 1])*dt/(dy**2)
for k in tqdm(range(nt-1)):
    T[k+1, 1:-1, 1:-1] += Imask * (1+stencilx[1]+stencily[1]) * T[k, 1:-1, 1:-1]
    T[k+1, 1:-1, 1:-1] += Imask*stencily[0] * T[k, 0:-2, 1:-1]
    T[k+1, 1:-1, 1:-1] += Imask*stencily[2] * T[k, 2:, 1:-1]
    T[k+1, 1:-1, 1:-1] += Imask*stencilx[0] * T[k, 1:-1, 0:-2]
    T[k+1, 1:-1, 1:-1] += Imask*stencilx[2] * T[k, 1:-1, 2:]

In [ ]:
ax = plot_field(xmesh, ymesh, T[-1])
[ax.axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
[ax.axhline(y=yi, color="k", ls="dotted") for yi in ymesh]
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Space in $y$")
ax.set_title(r"Numerical solution $T_{num}$ at final time")

In [ ]:
del xmesh
del ymesh
del tmesh
del T

# Sistemas de Equações Lineares

### Teoria

Existem diversas formas de resolver sistemas lineares, mas eles se dividem em duas partes:

1. Métodos diretos

> $$[A] \cdot [X] = [B] \Rightarrow [X^{\star}] = [A]^{-1} \cdot [B]$$
> 
> Que o mais conhecido é através da elimininação gaussiana e suas variações.

2. Métodos iterativos

> A partir de um vetor $x_{0}$ inicial, calculamos sucessivamente novos vetor, que esperamos que se aproxime da solução exata $\left[X^{\star}\right]$.
> 
> $$\left[X_{k+1}\right] = \left[T\right] \cdot \left[X_{k}\right] + \left[C\right]$$
> 
> Com matrizes $\left[T\right]$ e $\left[C\right]$ que dependem do método utilizado.
> 
> Este método é muito parecido com o método do ponto fixo com uma variável, mas neste caso iteramos em vetores

Iremos nos atentar apenas nos métodos iterativos, e desta forma iremos nos aprofundar apenas neles. Para ele, seja $D$ a diagonal de $A$, $U$ a matriz triangular inferior de $A$ e $U$ a matriz triangular superior, então:

$$
[A] = \begin{bmatrix}
A_{11}&A_{12}&A_{13}\\
A_{21}&A_{22}&A_{23}\\
A_{31}&A_{32}&A_{33}
\end{bmatrix}
 = [D]+[L] +[U]
$$
$$
[D] = \begin{bmatrix}
A_{11} & 0 & 0 \\
0 & A_{22} & 0 \\
0 & 0 & A_{33}
\end{bmatrix}\ \  \ [L] = \begin{bmatrix}
0 & 0 & 0 \\
A_{21} & 0 & 0 \\
A_{31} & A_{32} & 0
\end{bmatrix}\
\ \ [U] = \begin{bmatrix}
0 & A_{12} & A_{13} \\
0 & 0 & A_{23} \\
0 & 0 & 0
\end{bmatrix}
$$

1. Método de Jacobi - [Link wikipedia](https://en.wikipedia.org/wiki/Jacobi_method)

> Temos 
>
> $$[A] \cdot [X] = [B]$$
> $$\left([D]+[L]+[U]\right) \cdot [X] = [B]$$
> $$[D] \cdot [X] = \left([L]+[U]\right) \cdot [X] + [B]$$
> $$[X] = \underbrace{[D]^{-1} \cdot \left([L]+[U]\right)}_{[T]} \cdot [X] + \underbrace{[D]^{-1} \cdot [B]}_{[C]}$$
> $$\begin{cases}[T] = [D]^{-1} \cdot \left([L]+[U]\right)\\ [C] = [D]^{-1} \cdot [B]\end{cases}$$
>
> Como a matriz $[D]$ é diagonal, então teremos que $[D]^{-1} = [D^{-1}] = \left[\frac{1}{A_{ii}}\right]$, o que é facil de se obter
>
> Na notação indicial temos
>
> $$[X_{k+1}]_{i} = \dfrac{1}{A_{ii}} \cdot \left(B_{i} - \sum_{\begin{smallmatrix}j=1\\ j\ne i\end{smallmatrix}}^{n} A_{ij} \cdot [X_{k}]_{j} \right)$$


In [ ]:
def Jacobi(A: np.ndarray, B: np.ndarray, X0: np.ndarray, atol: float, verbose = False) -> Tuple[np.ndarray, int]:
    n = len(B)
    iteration = 0
    itermax = 200
    Xnew = np.copy(X0)
    while True:
        if verbose:
            print(f"X[{iteration}] = ", Xnew)
        for i in range(n):
            Xnew[i] = B[i]
            Xnew[i] -= sum(A[i,:i]*X0[:i])
            Xnew[i] -= sum(A[i,i+1:]*X0[i+1:])
            Xnew[i] /= A[i, i]
        error = np.max(np.abs(Xnew-X0))
        if verbose:
            print("    error = %.2e" % error)
        if error < atol:
            return Xnew, iteration
        X0 = np.copy(Xnew)
        iteration += 1
        if iteration > itermax:
            error_msg = f"Jacobi doesn't converge."
            raise ValueError(error_msg)

2. Método de Gauss-Seidel - [Link wikipedia](https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method)

> Temos 
>
> $$[A] \cdot [X] = [B]$$
> $$\left([D]+[L]+[U]\right) \cdot [X] = [B]$$
> $$\left([D]+[L]\right) \cdot [X] =   [U] \cdot [X] + [B]$$
> $$[X] = \underbrace{\left([D]+[L]\right)^{-1} \cdot [U]}_{[T]} \cdot [X] + \underbrace{\left([D]+[L]\right)^{-1} \cdot [B]}_{[C]}$$
> $$\begin{cases}[T] = \left([D]+[L]\right)^{-1} \cdot [U] \\ [C] = \left([D]+[L]\right)^{-1} \cdot [B]\end{cases}$$
>
> A inversa de $[D]+[L]$ é facil de conseguir pois a matriz é triangular e se pode obter $[M]$:
>
> $$\left([D]+[L]\right)^{-1} = \begin{bmatrix}A_{11} & 0 & \cdots & 0 \\ A_{21} & A_{22} & \cdots & 0 \\ \vdots & \vdots & \ddots & \vdots \\ A_{n1} & A_{n2} & \cdots & A_{nn}\end{bmatrix}^{-1} = \begin{bmatrix}M_{11} & 0 & \cdots & 0 \\ M_{21} & M_{22} & \cdots & 0 \\ \vdots & \vdots & \ddots & \vdots \\ M_{n1} & M_{n2} & \cdots & M_{nn}\end{bmatrix} = [M]$$
>
> Pode-se obter o mesmo resultado através da iteração utilizando a substituição progressiva:
>
> $$[X_{k+1}]_{i} = \dfrac{1}{A_{ii}} \cdot \left(B_{i} - \sum_{j=1}^{i-1} A_{ij} \cdot [X_{k+1}]_{j} -  \sum_{j=i+1}^{n} A_{ij} \cdot [X_{k}]_{j} \right)$$

In [ ]:
def GaussSeidel(A: np.ndarray, B: np.ndarray, X0: np.ndarray, atol: float, verbose = False) -> Tuple[np.ndarray, int]:
    n = len(B)
    iteration = 0
    itermax = 200
    Xnew = np.copy(X0)
    while True:
        if verbose:
            print(f"X[{iteration}] = ", Xnew)
        for i in range(n):
            Xnew[i] = B[i]
            Xnew[i] -= sum(A[i,:i]*Xnew[:i])
            Xnew[i] -= sum(A[i,i+1:]*X0[i+1:])
            Xnew[i] /= A[i, i]
        error = np.max(np.abs(Xnew-X0))
        if verbose:
            print("    error = %.2e" % error)
        if error < atol:
            return Xnew, iteration
        X0 = np.copy(Xnew)
        iteration += 1
        if iteration > itermax:
            error_msg = f"Gauss Seidel doesn't converge."
            raise ValueError(error_msg)

3. Método SOR - [Link wikipedia](https://en.wikipedia.org/wiki/Successive_over-relaxation)

> Temos 
>
> $$[A] \cdot [X] = [B]$$
> $$\left([D]+[L]+[U]\right) \cdot [X] = [B]$$
> $$\omega \left([D]+[L]+[U]\right) \cdot [X] = \omega \cdot [B]$$
> $$\omega [L] \cdot [X] = \omega [B] -\left(\omega [U] + \omega[D]\right) \cdot [X]$$
> $$\left([D]+\omega [L] \right)\cdot [X] = \omega [B] - \left(\omega[U] + (\omega-1)[D]\right)\cdot [X]$$
> $$[X] = \underbrace{-\left([D]+\omega [L]\right)^{-1} \cdot \left(\omega[U]+(\omega-1)[D]\right)}_{[T]} \cdot [X] + \underbrace{\left([D]+\omega [L]\right)^{-1} \cdot [B]}_{[C]}$$
> $$\begin{cases}[T] = -\left([D]+\omega [L]\right)^{-1} \cdot \left(\omega[U]+(\omega-1)[D]\right) \\ [C] = \left([D]+\omega [L]\right)^{-1} \cdot [B]\end{cases}$$
>
> Pode-se obter o mesmo resultado através da iteração utilizando a substituição progressiva:
>
> $$[X_{k+1}]_{i} = (1-\omega) [X_{k}]_{i}+ \dfrac{\omega}{A_{ii}} \cdot \left(B_{i} - \sum_{j=1}^{i-1} A_{ij} \cdot [X_{k+1}]_{j} -  \sum_{j=i+1}^{n} A_{ij} \cdot [X_{k}]_{j} \right)$$

In [ ]:
def SORMethod(A: np.ndarray, B: np.ndarray, X0: np.ndarray, omega: float, atol: float, verbose = False) -> Tuple[np.ndarray, int]:
    n = len(B)
    iteration = 0
    itermax = 200
    Xnew = np.copy(X0)
    while True:
        if verbose:
            print(f"X[{iteration}] = ", Xnew)
        for i in range(n):
            Xnew[i] = B[i]
            Xnew[i] -= sum(A[i,:i]*Xnew[:i])
            Xnew[i] -= sum(A[i,i+1:]*X0[i+1:])
            Xnew[i] *= omega/A[i, i]
            Xnew[i] += (1-omega)*X0[i]
        error = np.max(np.abs(Xnew-X0))
        if verbose:
            print("    error = %.2e" % error)
        if error < atol:
            return Xnew, iteration
        X0 = np.copy(Xnew)
        iteration += 1
        if iteration > itermax:
            error_msg = f"SOR Method doesn't converge."
            raise ValueError(error_msg)

4. Gradiente Conjulgado - [Link wikipedia](https://en.wikipedia.org/wiki/Conjugate_gradient_method)

> Este método é utilizado partindo da ideia que resolver o sistema linear $[A]\cdot [X] = [B]$ significa que queremos encontrar o valor de $[X]$ tal que minimize a função
>
> $$J([X]) = \dfrac{1}{2}[X]^{T} \cdot [A] \cdot [X] - [X]^{T} \cdot [B]$$
> 
> De tal forma que a derivada de $J$ em relação a $X$ significa:
>
> $$\dfrac{dJ}{dX} = A \cdot X - B $$
> 
> A ideia deste método consiste em dizer 

In [ ]:
def GradienteConjugado(A: np.ndarray, B: np.ndarray, X0: np.ndarray, omega: float, atol: float, verbose=False) -> Tuple[np.ndarray, int]:
    n = len(B)
    iteration = 0
    itermax = 200
    r0 = B - A @ X0
    p0 = r0
    Xnew = np.copy(X0)
    while True:
        if verbose:
            print(f"X[{iteration}] = ", Xnew)
        alpha = np.inner(r0, r0)/(p0 @ A @ p0)
        Xnew = X0 + alpha * p0
        rnew = r0 - alpha * A @ p0
        error = np.max(np.abs(Xnew-X0))
        if verbose:
            print("    error = %.2e" % error)
        if error < atol:
            return Xnew, iteration
        beta = np.inner(rnew, rnew)/np.inner(r0, r0)
        p0 = rnew + beta * p0
        r0 = np.copy(rnew)
        iteration += 1
        if iteration > itermax:
            error_msg = f"Gradiente conjulgado doesn't converge."
            raise ValueError(error_msg)

### Exercicio 10

Use o método de Gauss-Seidel para resolver o seguinte sistema:

$$
\begin{bmatrix}
12 & -2 & 3 & -1 \\
1 & 6 & 20 & -4 \\
-2 & 15 & 6 & -3 \\
0 & -3 & 2 & 9
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3\\
x_4
\end{bmatrix}=
\begin{bmatrix}
0 \\
20\\
0\\
0
\end{bmatrix}
$$

A solução deste sistema é dada por

$$
X^{\star} =
\dfrac{1}{2653}
\begin{bmatrix}
-1092 \\ -1556 \\ 2940 \\ -1172
\end{bmatrix}
\approx 
\begin{bmatrix}
-0.4116  \\ -5865 \\ 1.1082 \\ -0.4418
\end{bmatrix}
$$

In [ ]:
A = [[12, -2,  3, -1],
     [ 1,  6, 20, -4],
     [-2, 15,  6, -3],
     [ 0, -3,  2,  9]]
B = [0, 20, 0, 0]
A = np.array(A, dtype="float64")
B = np.array(B, dtype="float64")
Xstar = np.array([-1092, -1556, 2940, -1172], dtype="float64")/2653
print(f"norm L2 (A * Xstar - B) = {np.linalg.norm(A@Xstar-B):.2e}")

Podemos ver que o método diretamente não resolve e diverge:

In [ ]:
try:
    X0 = np.random.uniform(-1, 1, B.shape)
    Xf, iter = GaussSeidel(A, B, X0, 1e-9, verbose=True)
except ValueError as e:
    print("Nao foi possivel encontrar a solucao do sistema.")
    print("Com a condicao iniciao X0 = ", X0)
    print("    Erro = ", e)

Para resolvermos, iremos fazer um pré-condicionamento e trocar as linhas $2$ e $3$ para obter:

$$
\begin{bmatrix}
12 & -2 & 3 & -1 \\
-2 & 15 & 6 & -3 \\
1 & 6 & 20 & -4 \\
0 & -3 & 2 & 9
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3\\
x_4
\end{bmatrix}=
\begin{bmatrix}
0 \\
0\\
20\\
0
\end{bmatrix}
$$

In [ ]:
A = [[12, -2,  3, -1],
     [-2, 15,  6, -3],
     [ 1,  6, 20, -4],
     [ 0, -3,  2,  9]]
B = [0, 0, 20, 0]
A = np.array(A, dtype="float64")
B = np.array(B, dtype="float64")
X0 = np.random.uniform(-1, 1, B.shape)
Xf, iter = GaussSeidel(A, B, X0, 1e-9, verbose=True)
print("Final solution = ")
print("    ", Xf)
print("Exact solution = ")
print("    ", Xstar)

# Equação do Calor 2D (Permanente)

### Exercício 12

In [ ]:
xmin, xmax = 0, 1
ymin, ymax = 0, 1
nx, ny = 10, 12
dx = (xmax-xmin)/(nx-1)
dy = (ymax-ymin)/(ny-1)
xmesh = np.linspace(xmin, xmax, nx)
ymesh = np.linspace(ymin, ymax, ny)
T = np.empty((ny, nx), dtype="object")
K = np.zeros((ny, nx, ny, nx))
F = np.zeros((ny, nx), dtype="float64")
stencil = np.array([[0, dx/dy, 0],
                    [dy/dx, -2*(dy/dx+dx/dy), dy/dx],
                    [0, dx/dy, 0]], dtype="float64")
T[:, 0] = 0  # x = 0
T[:, nx-1] = 0  # x = 1
T[0, :] = 0  # y = 0
T[ny-1, :] = 1  # y = 1

for i in range(1, nx-1):
    for j in range(1, ny-1):
        K[j, i, j-1:j+2, i-1:i+2] = stencil[:, :]

mexp = (T == None).flatten()
Kexp = K.reshape((ny*nx, ny*nx))
Fexp = F.reshape(ny*nx)
Texp = T.reshape(ny*nx)
Kuu = np.delete(np.delete(Kexp, ~mexp, axis=0), ~mexp, axis=1)
Kku = np.delete(np.delete(Kexp, mexp, axis=0), ~mexp, axis=1)
Kuk = np.delete(np.delete(Kexp, ~mexp, axis=0), mexp, axis=1)
Kkk = np.delete(np.delete(Kexp, mexp, axis=0), mexp, axis=1)
Texp[mexp] = np.linalg.solve(Kuu, (Fexp[mexp] - Kuk @ Texp[~mexp]).astype("float64"))
Fexp[~mexp] = Kku @ Texp[mexp] + Kkk @ Texp[~mexp]
T = Texp.reshape((ny, nx))
F = Fexp.reshape((ny, nx))

In [ ]:
ax = plot_field(xmesh, ymesh, T)
[ax.axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
[ax.axhline(y=yi, color="k", ls="dotted") for yi in ymesh]
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Space in $y$")
ax.set_title(r"Numerical solution $T_{num}$ at final time")

### Exercicio 13

In [ ]:
xmin, xmax = 0, 1
ymin, ymax = 0, 1
nx, ny = 65, 71
dx = (xmax-xmin)/(nx-1)
dy = (ymax-ymin)/(ny-1)
xmesh = np.linspace(xmin, xmax, nx)
ymesh = np.linspace(ymin, ymax, ny)
T = np.empty((ny, nx), dtype="object")
K = np.zeros((ny, nx, ny, nx))
F = np.zeros((ny, nx), dtype="float64")
stencil = np.array([[0, dx/dy, 0],
                    [dy/dx, -2*(dy/dx+dx/dy), dy/dx],
                    [0, dx/dy, 0]], dtype="float64")
T[:, 0] = 0  # x = 0
T[:, nx-1] = 0  # x = 1
T[0, :] = 0  # y = 0
T[ny-1, :] = 0  # y = 1

for i in range(1, nx-1):
    for j in range(1, ny-1):
        K[j, i, j-1:j+2, i-1:i+2] = stencil[:, :]
F[:, :] = -2*np.pi**2 *dy*dx* np.tensordot(np.sin(np.pi*ymesh), np.sin(np.pi*xmesh), axes=0)

def solve_4D_system(K: np.ndarray, F: np.ndarray, T: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    mexp = (T == None).flatten()
    Kexp = K.reshape((ny*nx, ny*nx))
    Fexp = F.reshape(ny*nx)
    Texp = T.reshape(ny*nx)
    Kuu = np.delete(np.delete(Kexp, ~mexp, axis=0), ~mexp, axis=1)
    Kku = np.delete(np.delete(Kexp, mexp, axis=0), ~mexp, axis=1)
    Kuk = np.delete(np.delete(Kexp, ~mexp, axis=0), mexp, axis=1)
    Kkk = np.delete(np.delete(Kexp, mexp, axis=0), mexp, axis=1)
    Texp[mexp] = np.linalg.solve(Kuu, (Fexp[mexp] - Kuk @ Texp[~mexp]).astype("float64"))
    Fexp[~mexp] = Kku @ Texp[mexp] + Kkk @ Texp[~mexp]
    T = Texp.reshape((ny, nx))
    F = Fexp.reshape((ny, nx))
    return T, F

T, F = solve_4D_system(K, F, T)

In [ ]:
Tanalitic = np.tensordot(np.sin(np.pi*ymesh), np.sin(np.pi*xmesh), axes=0)

In [ ]:
ax = plot_field(xmesh, ymesh, T)
[ax.axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
[ax.axhline(y=yi, color="k", ls="dotted") for yi in ymesh]
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Space in $y$")
ax.set_title(r"Numerical solution $T_{num}$ at final time")

In [ ]:
diff = np.abs(T-Tanalitic)
ax = plot_field(xmesh, ymesh, diff)
[ax.axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
[ax.axhline(y=yi, color="k", ls="dotted") for yi in ymesh]
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Space in $y$")
ax.set_title(r"Error")

# Diferentes formulações: Explicito, Implicito e Crank-Nicolson

### Exercício 14

> Resolva novamente o problema proposto no exercício 3, mas agora com o Método BTCS.
> 
> $$\dfrac{T_{k+1,i}-T_{k,i}}{h_t} = \dfrac{T_{k+1,i-1}-2T_{k+1,i}+T_{k+1,i+1}}{h_x^2}$$
> 
> e depois com o método de Crank-Nicolson:
> 
> $$\dfrac{T_{k+1,i}-T_{k,i}}{h_t} = \dfrac{1}{2}\left(\dfrac{T_{k,i-1}-2T_{k,i}+T_{k,i+1}}{h_x^2} + \dfrac{T_{k+1,i-1}-2T_{k+1,i}+T_{k+1,i+1}}{h_x^2}\right)$$
>
> Compare os resultados obtidos com os 3 métodos numéricos com a solução analítica.
>
> > **Exercicio 3**
> > $$\dfrac{\partial T}{\partial t} = \dfrac{\partial^2 T}{\partial x^2}$$
> >
> > $$\Omega = \underbrace{\left[0, \ 1\right]}_{x} \times \underbrace{\left[0, \ \infty\right)}_{t}$$
> >
> > $$T(0, \ t) = 1 \ \ \ \ \ \forall 0 \le t$$
> > $$T(1, \ t) = 0 \ \ \ \ \ \forall 0 \le t$$
> > $$T(x, \ 0) = 0 \ \ \ \ 0 < x < 1$$
> > 
> > $$T_{exact}(x, \ t) = 1-x-\sum_{n=1}^{\infty} \dfrac{2}{n\pi} \cdot \exp \left(-n^2\pi^2 t\right) \cdot \sin \left(n\pi x\right)$$



In [ ]:
xmin, xmax = 0, 1
tmin, tmax = 0, 0.3
nx, nt = 51, 1497
dx = (xmax-xmin)/(nx-1)
dt = (tmax-tmin)/(nt-1)
xmesh = np.linspace(xmin, xmax, nx)
tmesh = np.linspace(tmin, tmax, nt)

In [ ]:
Tanalitic = np.zeros((nt, nx), dtype="float64")
for n in range(1, 21):
    exppart = np.exp(-n**2*(np.pi**2)*tmesh)
    sinpart = np.sin(n*np.pi*xmesh)
    Tanalitic -= np.tensordot(exppart, sinpart, axes=0)/n
Tanalitic *= 2/np.pi
for i, xi in enumerate(xmesh):
    Tanalitic[:, i] += 1-xi

In [ ]:
Texplicit = np.zeros((nt, nx), dtype="float64")
Texplicit[:, 0] = 1
a = dt/dx**2
for k in tqdm(range(nt-1)):
    Texplicit[k+1, 1:-1] = (1-2*a)*Texplicit[k, 1:-1]
    Texplicit[k+1, 1:-1] += a * Texplicit[k, 0:-2]
    Texplicit[k+1, 1:-1] += a * Texplicit[k, 2:]

Para o método implícito BTCS, teremos

> $$\dfrac{{\color{red}{T_{k+1,i}}}-T_{k,i}}{h_t} = \dfrac{{\color{red}{T_{k+1,i-1}}}-2{\color{red}{T_{k+1,i}}}+{\color{red}{T_{k+1,i+1}}}}{h_x^2}$$
> 
> Que não conhecemos os valores em vermelho, de forma que podemos montar:
> 
> $$\begin{bmatrix}
\dfrac{-1}{h_x^2}& \left(\dfrac{2}{h_x^2} + \dfrac{1}{h_t}\right) & \dfrac{-1}{h_x^2}
\end{bmatrix}\begin{bmatrix}
{\color{red}{T_{k+1,i-1}}} \\
{\color{red}{T_{k+1,i}}} \\
{\color{red}{T_{k+1,i+1}}} \\
\end{bmatrix} = \dfrac{T_{k,i}}{h_t}$$
> 
> Sendo essa equação válida para qualquer $i = 1, \ 2, \ \cdots, \ n_x - 2$, e adicionando a condição de contorno então temos o sistema linear:
> $$\begin{bmatrix}
1 & & & & \cdots & & \\
\frac{-1}{h_x^2}& \left(\frac{2}{h_x^2} + \frac{1}{h_t}\right) & \frac{-1}{h_x^2} & & \cdots & & \\
& \frac{-1}{h_x^2}& \left(\frac{2}{h_x^2} + \frac{1}{h_t}\right) & \frac{-1}{h_x^2} & \cdots & & \\
\vdots & \vdots & \ddots & \ddots & \ddots & \vdots & \vdots  \\
& & \cdots & \frac{-1}{h_x^2} & \left(\frac{2}{h_x^2} + \frac{1}{h_t}\right) & \frac{-1}{h_x^2} &  \\
& & \cdots & & \frac{-1}{h_x^2} & \left(\frac{2}{h_x^2} + \frac{1}{h_t}\right) & \frac{-1}{h_x^2} \\
& & \cdots & & & & 1
\end{bmatrix}
\begin{bmatrix}
T_{k+1,0} \\ T_{k+1,1} \\ T_{k+1,2} \\ \vdots \\ T_{k+1,n-3} \\ T_{k+1, n-2} \\ T_{k+1,n-1}
\end{bmatrix} = 
\begin{bmatrix}
T_a \\ \frac{T_{k,1}}{h_t} \\ \frac{T_{k,2}}{h_t} \\ \vdots \\ \frac{T_{k,n-3}}{h_t} \\ \frac{T_{k,n-2}}{h_t} \\ T_b
\end{bmatrix}$$
> 
> Que podemos reescrever, com $a=\frac{1}{2+\frac{h_x^2}{h_t}}$ e $b = \frac{1}{1+\frac{2h_t}{h_x^2}}$:
> 
> $$\begin{bmatrix}
1 & -a & & \cdots & \\
-a & 1 & -a & \cdots & \\
 \vdots & \ddots & \ddots & \ddots & \vdots \\
 & \cdots & -a & 1 & -a  \\
& \cdots & & -a & 1 \\
\end{bmatrix}
\begin{bmatrix}
T_{k+1,0} \\ T_{k+1,1} \\ \vdots \\ T_{k+1,n-3} \\ T_{k+1, n-2} 
\end{bmatrix} = 
b \cdot
\begin{bmatrix}
T_{k,1} \\ T_{k,2} \\ \vdots \\ T_{k,n-3} \\ T_{k,n-2}
\end{bmatrix} + a \cdot \begin{bmatrix}T_a \\ 0 \\ \vdots \\ 0 \\ T_b \end{bmatrix}$$
> 
> Tal sistema é tridiagonal, simétrico e e é facil de ser resolvido utilizando um método direto de eliminação gaussiana:
> $$\begin{bmatrix}
\square & \circ & \cdots & & \\
\circ & \square  & \cdots & & \\
\vdots &  \vdots & \ddots & \vdots & \vdots \\
& & \cdots & \square & \circ \\
& & \cdots & \circ & \square
\end{bmatrix} \Rightarrow \begin{bmatrix}
\square & \circ & \cdots & & \\
  & \square  & \cdots & & \\
\vdots &  \vdots & \ddots & \vdots & \vdots \\
& & \cdots & \square & \circ \\
& & \cdots &   & \square
\end{bmatrix} \Rightarrow \begin{bmatrix}
\square &   & \cdots & & \\
  & \square  & \cdots & & \\
\vdots &  \vdots & \ddots & \vdots & \vdots \\
& & \cdots & \square &   \\
& & \cdots &   & \square
\end{bmatrix}$$

In [ ]:
Timplicit = np.zeros((nt, nx), dtype="float64")
Timplicit[:, 0] = 1
a = 1/(2+dx**2/dt)
b = 1/(1+2*dt/dx**2)
diagonalvec = np.ones(nx)
for k in tqdm(range(nt-1)):
    diagonalvec[:] = 1
    Timplicit[k+1, 1:-1] = b*Timplicit[k, 1:-1]
    Timplicit[k+1, 1] += a*Timplicit[k+1, 0]
    Timplicit[k+1, -2] += a*Timplicit[k+1, -1]
    for i in range(2, nx-1):
        diagonalvec[i] -= a**2 / diagonalvec[i-1]
        Timplicit[k+1, i] += a*Timplicit[k+1, i-1]/diagonalvec[i-1]
    Timplicit[k+1, -2] /= diagonalvec[-2]
    for i in range(nx-3, 0, -1):
        Timplicit[k+1, i] += a*Timplicit[k+1, i+1]
        Timplicit[k+1, i] /= diagonalvec[i]

Para o método misto de Crank-Nicolson, teremos

> $$\begin{bmatrix}
\dfrac{-1}{2h_x^2}& \left(\dfrac{1}{h_t}+\dfrac{1}{h_x^2}\right) & \dfrac{-1}{2h_x^2}
\end{bmatrix}\begin{bmatrix}
{\color{red}{T_{k+1,i-1}}} \\
{\color{red}{T_{k+1,i}}} \\
{\color{red}{T_{k+1,i+1}}} \\
\end{bmatrix} = \begin{bmatrix}
\dfrac{1}{2h_x^2}& \left(\dfrac{1}{h_t} - \dfrac{1}{h_x^2}\right) & \dfrac{1}{2h_x^2}
\end{bmatrix}\begin{bmatrix}
T_{k,i-1} \\
T_{k,i} \\
T_{k,i+1} \\
\end{bmatrix} $$
> 
> Sendo essa equação válida para qualquer $i = 1, \ 2, \ \cdots, \ n_x - 2$, adicionando a condição de contorno, e reduzindo o sistema linear teremos:
> 
> $$\left[\begin{smallmatrix}
1 & -a & & \\
-a & 1 & -a & \\
&  \ddots & \ddots & \ddots  &  \\
& & -a & 1 & -a  \\
& & & -a & 1
\end{smallmatrix}\right]
\left[\begin{smallmatrix}
T_{k+1,1} \\ T_{k+1,2} \\ \vdots \\ T_{k+1,n-3} \\ T_{k+1, n-2}
\end{smallmatrix}\right] =
\left[\begin{smallmatrix}
a & b & a  & & & \\
& a & b & a  & & \\
& & \ddots & \ddots & \ddots  & & \\
& & & a & b & a  &  \\
& & & & a & b & a 
\end{smallmatrix}\right]
\left[\begin{smallmatrix}
T_{k,0} \\ T_{k,1} \\ \vdots \\ T_{k,n-2} \\ T_{k, n-1}
\end{smallmatrix}\right] + a\left[\begin{smallmatrix}
T_{a} \\ 0 \\ \vdots \\ 0 \\ T_b
\end{smallmatrix}\right]$$
> 
> Com 
> 
> $$a = \dfrac{\left(\dfrac{1}{2h_x^2}\right)}{\left(\dfrac{1}{h_t}+\dfrac{1}{h_x^2}\right)} = \dfrac{h_t}{2(h_t+h_x^2)}; \ \ \ \ \ \ \ \ \ b =\dfrac{\left(\dfrac{1}{h_t}-\dfrac{1}{h_x^2}\right)}{\left(\dfrac{1}{h_t}+\dfrac{1}{h_x^2}\right)} = \dfrac{h_x^2 - h_t}{h_x^2 + h_t}$$
> 
> Tal sistema se resolve igual ao sistema do esquema implícito BTCS, pois é tridiagonal

In [ ]:
Tcranknico = np.zeros((nt, nx), dtype="float64")
Tcranknico[:, 0] = 1
a = dt/(2*(dt+dx**2))
b = (dx**2-dt)/(dx**2+dt)
diagonalvec = np.ones(nx)
for k in tqdm(range(nt-1)):
    diagonalvec[:] = 1
    Tcranknico[k+1, 1:-1] = a*Tcranknico[k, :-2]
    Tcranknico[k+1, 1:-1] += b*Tcranknico[k, 1:-1]
    Tcranknico[k+1, 1:-1] += a*Tcranknico[k, 2:]
    Tcranknico[k+1, 1] += a*Tcranknico[k+1, 0]
    Tcranknico[k+1, -2] += a*Tcranknico[k+1, -1]
    for i in range(2, nx-1):
        diagonalvec[i] -= a**2 / diagonalvec[i-1]
        Tcranknico[k+1, i] += a*Tcranknico[k+1, i-1]/diagonalvec[i-1]
    Tcranknico[k+1, -2] /= diagonalvec[-2]
    for i in range(nx-3, 0, -1):
        Tcranknico[k+1, i] += a*Tcranknico[k+1, i+1]
        Tcranknico[k+1, i] /= diagonalvec[i]

In [ ]:
fig, axis = plt.subplots(1, 3, figsize=(30, 10))
plot_field(xmesh, tmesh, Texplicit, axis[0])
[axis[0].axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
axis[0].set_title("Metodo explicito")
axis[0].set_xlabel("Space in $x$")
axis[0].set_ylabel("Time $t$")
plot_field(xmesh, tmesh, Timplicit, axis[1])
[axis[1].axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
axis[1].set_title("Metodo implicito BTCS")
axis[1].set_xlabel("Space in $x$")
axis[1].set_ylabel("Time $t$")
plot_field(xmesh, tmesh, Tcranknico, axis[2])
[axis[2].axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
axis[2].set_title("Metodo misto Crank-Nicolson")
axis[2].set_xlabel("Space in $x$")
axis[2].set_ylabel("Time $t$")
plt.suptitle(r"Valor da solução numerica $T_{num}$",fontsize=20)

In [ ]:
fig, axis = plt.subplots(1, 3, figsize=(30, 10))
plot_field(xmesh, tmesh, np.abs(Texplicit-Tanalitic), axis[0])
[axis[0].axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
axis[0].set_title("Metodo explicito")
axis[0].set_xlabel("Space in $x$")
axis[0].set_ylabel("Time $t$")
plot_field(xmesh, tmesh, np.abs(Timplicit-Tanalitic), axis[1])
[axis[1].axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
axis[1].set_title("Metodo implicito BTCS")
axis[1].set_xlabel("Space in $x$")
axis[1].set_ylabel("Time $t$")
plot_field(xmesh, tmesh, np.abs(Tcranknico-Tanalitic), axis[2])
[axis[2].axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
axis[2].set_title("Metodo misto Crank-Nicolson")
axis[2].set_xlabel("Space in $x$")
axis[2].set_ylabel("Time $t$")
plt.suptitle(r"Erro da solução numerica $|T_{num}-T_{ana}|$",fontsize=20)

In [ ]:
del xmesh
del tmesh
del Texplicit
del Timplicit
del Tcranknico

### Exercício 15

> Resolva novamente o problema proposta no exercício 5, mas agora com os métodos BTCS e Crank-Nicolson. Compare as soluções
> > **Exercicio 5**
> > $$\dfrac{\partial T}{\partial t} = 0.01\cdot \dfrac{\partial^2 T}{\partial x^2}$$
> > $$\Omega = \underbrace{\left[0, \ 0.5\right]}_{x} \times \underbrace{\left[0, \ \infty\right)}_{t}$$
> > $$T(0, \ t) = 0 \ \ \ \ \ \forall \ 0 \le t$$
> > $$\left[\dfrac{\partial T}{\partial x}\right]_{x=0.5}= 0 \ \ \ \ \ \forall \ 0 \le t$$
> > $$T(x, \ 0) = 200x \ \ \ \ \ \forall  \ 0 \le x \le 0.5$$
> > $$T_{exact}(x, \ t) = \dfrac{800}{\pi^2}\sum_{n=0}^{\infty} \dfrac{(-1)^{n}}{(2n+1)^2} \cdot \exp \left(-(2n+1)^2\pi^2 \cdot 0.01 \cdot t\right) \cdot \sin \left((2n+1)\pi x\right)$$


Da mesma forma que fizemos na Questão 14, mas agora temos a formulação com um parâmetro $\alpha \in \left[0, \ 1\right] \sub \mathbb{R}$:

* Método explícito: $\alpha = 0$
* Método implícito: $\alpha = 1$
* Método misto Crank-Nicolson: $\alpha = 0.5$

> $$\dfrac{T_{k+1,i}-T_{k,i}}{h_t} = \dfrac{1-\alpha}{h_x^2} \left(T_{k,i-1}-2T_{k,i}+T_{k,i+1}\right) + \dfrac{\alpha}{h_x^2}\left(T_{k+1,i-1}-2T_{k+1,i}+T_{k+1,i+1}\right)$$
> 
> $$\begin{bmatrix}
\dfrac{-\alpha}{h_x^2}& \left(\dfrac{1}{h_t}+\dfrac{2\alpha}{h_x^2}\right) & \dfrac{-\alpha}{h_x^2}
\end{bmatrix}\begin{bmatrix}
T_{k+1,i-1} \\
T_{k+1,i} \\
T_{k+1,i+1} \\
\end{bmatrix} = \begin{bmatrix}
\dfrac{1-\alpha}{h_x^2}& \left(\dfrac{1}{h_t} - \dfrac{2(1-\alpha)}{h_x^2}\right) & \dfrac{1-\alpha}{h_x^2}
\end{bmatrix}\begin{bmatrix}
T_{k,i-1} \\
T_{k,i} \\
T_{k,i+1} \\
\end{bmatrix} $$
> 
> Que reescrevendo com parâmetros
> 
> $$a = \dfrac{2h_t}{2\alpha h_t + h_x^2} \ \ \ \ \ \ \ \ b = \dfrac{h_x^2-2(1-\alpha)h_t}{h_x^2 + 2\alpha h_t}$$
> $$\begin{bmatrix}
-\alpha a & 1 & -\alpha a
\end{bmatrix}\begin{bmatrix}
T_{k+1,i-1} \\
T_{k+1,i} \\
T_{k+1,i+1} \\
\end{bmatrix} = \begin{bmatrix}
(1-\alpha) a & b & (1-\alpha)a
\end{bmatrix}\begin{bmatrix}
T_{k,i-1} \\
T_{k,i} \\
T_{k,i+1} \\
\end{bmatrix} $$


A diferença principal se dá na condição de Neumann em $x=0.5$, que de mesmo modo feito no Exercício 5:

$$
\dfrac{\partial T}{\partial x} = 0 \Rightarrow \dfrac{T_{k,n}-T_{k,n-2}}{2h_x} = 0 \Rightarrow T_{k,n}=T_{k,n-2}
$$

E aplicando para a formulação matricial em $i=n_x-1$ tem-se 

$$\begin{bmatrix}
-2\alpha a & 1
\end{bmatrix}\begin{bmatrix}
T_{k+1,n_x-2} \\
T_{k+1,n_x-1}
\end{bmatrix} = \begin{bmatrix}
2(1-\alpha) a & b
\end{bmatrix}\begin{bmatrix}
T_{k,n_x-2} \\
T_{k,n_x-1}
\end{bmatrix} $$

Então montando o sistema total

$$
\left[
\begin{smallmatrix}
1 & -\alpha a & & & \\
 -\alpha a & 1 & -\alpha a & & \\
& \ddots & \ddots & \ddots & \\
& & -\alpha a & 1 & -\alpha a \\
& & & -2\alpha a & 1
\end{smallmatrix}\right]
\left[\begin{smallmatrix}
T_{k+1, \ 1} \\ T_{k+1, \ 2} \\ \vdots \\ T_{k+1, \ n_x-2} \\ T_{k+1, \ n_x-1}
\end{smallmatrix}\right]
=
\left[
\begin{smallmatrix}
b & (1-\alpha) a & & & \\
(1-\alpha) a & b & (1-\alpha) a & & \\
& \ddots & \ddots & \ddots & & \\
& & (1-\alpha) a & b & (1-\alpha) a \\
& & & 2(1-\alpha) a & b
\end{smallmatrix}\right]
\left[\begin{smallmatrix}
T_{k, \ 1} \\ \vdots \\ T_{k, \ n_x-2} \\ T_{k, \ n_x-1}
\end{smallmatrix}\right]+
\left[\begin{smallmatrix}
\alpha a T_{k+1,0} + (1-\alpha) a T_{k,0} \\ 0  \\ \vdots \\ 0 \\ 0
\end{smallmatrix}\right]
$$



In [ ]:
xmin, xmax = 0, 0.5
tmin, tmax = 0, 3
nx, nt = 21, 20000
dx = (xmax-xmin)/(nx-1)
dt = (tmax-tmin)/(nt-1)
xmesh = np.linspace(xmin, xmax, nx)
tmesh = np.linspace(tmin, tmax, nt)    

In [ ]:
Tanalitic = np.zeros((nt, nx), dtype="float64")
Texplicit = np.zeros((nt, nx), dtype="float64")
Timplicit = np.zeros((nt, nx), dtype="float64")
Tcranknico = np.zeros((nt, nx), dtype="float64")
for i, xi in enumerate(xmesh):
    Texplicit[0, i] = 200*xi
    Timplicit[0, i] = 200*xi
    Tcranknico[0, i] = 200*xi
for n in range(0, 21):
    exppart = np.exp(-(2*n+1)**2*(np.pi**2)*0.01*tmesh)
    sinpart = np.sin((2*n+1)*np.pi*xmesh)
    Tanalitic += (-1)**n * np.tensordot(exppart, sinpart, axes=0)/((2*n+1)**2)
Tanalitic *= 800/(np.pi**2)

In [ ]:
const = 0.01*dt/(dx**2)
for k in tqdm(range(nt-1)):
    Texplicit[k+1, nx-1] = 2*const*Texplicit[k, nx-2] + (1-2*const)*Texplicit[k,nx-1]
    Texplicit[k+1, 1:nx-1] += (1-2*const)*Texplicit[k,1:nx-1]
    Texplicit[k+1, 1:nx-1] += const*Texplicit[k, :nx-2]
    Texplicit[k+1, 1:nx-1] += const*Texplicit[k, 2:]


In [ ]:
alpha = 1
a = 1/(2*alpha+1/const)
b = (1-2*(1-alpha)*const)/(2*alpha*const+1)
Mimplicit = np.eye(nx-1, dtype="float64")
for i in range(nx-2):
    Mimplicit[i,i+1] = -alpha*a
    Mimplicit[i+1,i] = -alpha*a
Mimplicit[-1, -2] += -alpha*a
Mimpinv = np.linalg.inv(Mimplicit)

for k in tqdm(range(nt-1)):
    B = b * np.copy(Timplicit[k, 1:])
    B[:-1] += a*(1-alpha) * Timplicit[k, 2:]
    B[1:] += a*(1-alpha) * Timplicit[k, 1:-1]
    B[0] += a*(alpha*Timplicit[k+1, 0] + (1-alpha)*Timplicit[k, 0])
    Timplicit[k+1,1:] = Mimpinv @ B

In [ ]:
alpha = 0.5
a = 1/(2*alpha+1/const)
b = (1-2*(1-alpha)*const)/(2*alpha*const+1)
Mcranknico = np.eye(nx-1, dtype="float64")
for i in range(nx-2):
    Mcranknico[i,i+1] = -alpha*a
    Mcranknico[i+1,i] = -alpha*a
Mcranknico[-1, -2] += -alpha*a
Mcrainv = np.linalg.inv(Mcranknico)

for k in tqdm(range(nt-1)):
    B = b * np.copy(Tcranknico[k, 1:])
    B[:-1] += a*(1-alpha) * Tcranknico[k, 2:]
    B[1:] += a*(1-alpha) * Tcranknico[k, 1:-1]
    B[0] += a*(alpha*Tcranknico[k+1, 0] + (1-alpha)*Tcranknico[k, 0])
    Tcranknico[k+1,1:] = Mcrainv @ B

In [ ]:
ncurves = 5
indexs = [int(curveindex) for curveindex in np.linspace(0, nt-1, ncurves)]
plt.figure(figsize=(10, 4))
for j in indexs:
    pl = plt.plot(xmesh, Tanalitic[j, :], ls="dotted")
    color = pl[0].get_color()
    plt.plot(xmesh, Texplicit[j, :], label=f"t = {tmesh[j]:.2}", color=color)
plt.legend()

In [ ]:
fig, axis = plt.subplots(1, 3, figsize=(30, 10))
plot_field(xmesh, tmesh, Texplicit, axis[0])
[axis[0].axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
axis[0].set_title("Metodo explicito")
axis[0].set_xlabel("Space in $x$")
axis[0].set_ylabel("Time $t$")
plot_field(xmesh, tmesh, Timplicit, axis[1])
[axis[1].axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
axis[1].set_title("Metodo implicito BTCS")
axis[1].set_xlabel("Space in $x$")
axis[1].set_ylabel("Time $t$")
plot_field(xmesh, tmesh, Tcranknico, axis[2])
[axis[2].axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
axis[2].set_title("Metodo misto Crank-Nicolson")
axis[2].set_xlabel("Space in $x$")
axis[2].set_ylabel("Time $t$")
plt.suptitle(r"Valor da solução numerica $T_{num}$",fontsize=20)

In [ ]:
fig, axis = plt.subplots(1, 3, figsize=(30, 10))
plot_field(xmesh, tmesh, np.abs(Texplicit-Tanalitic), axis[0])
[axis[0].axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
axis[0].set_title("Metodo explicito")
axis[0].set_xlabel("Space in $x$")
axis[0].set_ylabel("Time $t$")
plot_field(xmesh, tmesh, np.abs(Timplicit-Tanalitic), axis[1])
[axis[1].axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
axis[1].set_title("Metodo implicito BTCS")
axis[1].set_xlabel("Space in $x$")
axis[1].set_ylabel("Time $t$")
plot_field(xmesh, tmesh, np.abs(Tcranknico-Tanalitic), axis[2])
[axis[2].axvline(x=xi, color="k", ls="dotted") for xi in xmesh]
axis[2].set_title("Metodo misto Crank-Nicolson")
axis[2].set_xlabel("Space in $x$")
axis[2].set_ylabel("Time $t$")
plt.suptitle(r"Erro da solução numerica $|T_{num}-T_{ana}|$",fontsize=20)

In [ ]:
del xmesh
del tmesh
del Texplicit
del Timplicit
del Tcranknico

### Exercício 16

> Resolva novamente o exercício 6, mas agora usando o método BTCS, e o Crank-Nicolson.
>
> Compare as 3 soluções numéricas entre si e com a solução analítica.
>
> > **Exercicio 6**
> > $$\dfrac{\partial T}{\partial t} = \dfrac{\partial^2 T}{\partial x^2}+\dfrac{\partial^2 T}{\partial y^2}$$
> > $$\Omega = \left[0,\ 1\right]\times\left[0,\ 1\right]\times\left[0,\ \infty\right)$$
> > $$\begin{align*}
T(x, \ 0, \ t) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(x, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right) \\
T(x, \ 1, \ t) & = \sin \left(\pi x\right) \ \ \ \ \ \forall \left(x, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right)  \\
T(0, \ y, \ t) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(y, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right) \\
T(1, \ y, \ t) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(y, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right)\\
T(x, \ y, \ 0) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(x, \ y\right) \in \left[0, \ 1\right]\times\left[0, \ 1\right]
\end{align*}$$
> > $$T_{analitic}(x, \ y,\ t\to \infty) = \dfrac{\sinh (\pi y) \sin (\pi x)}{\sinh \pi}$$


### Exercício 17

> Resolva o exercício 7 usando os métodos BTCS e Crank-Nicolson.
> 
> Compare as soluções.
> > **Exercicio 7**
> > $$\dfrac{\partial T}{\partial t} = \dfrac{\partial^2 T}{\partial x^2}+\dfrac{\partial^2 T}{\partial y^2}$$
> > $$\Omega = \left[0,\ 0.5\right]\times\left[0,\ 1\right]\times\left[0,\ \infty\right)$$
> > $$\begin{align*}
T(x, \ 0, \ t) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(x, \ t\right) \in \left[0, \ \frac{1}{2}\right]\times\left[0,\ \infty\right) \\
T(x, \ 1, \ t) & = \sin \left(\pi x\right) \ \ \ \ \ \forall \left(x, \ t\right) \in \left[0, \ \frac{1}{2}\right]\times\left[0,\ \infty\right)  \\
T(0, \ y, \ t) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(y, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right) \\
\dfrac{\partial T}{\partial x}\left(\frac{1}{2}, \ y, \ t\right) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(y, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right)\\
T(x, \ y, \ 0) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(x, \ y\right) \in \left[0, \ \frac{1}{2}\right]\times\left[0, \ 1\right]
\end{align*}$$
> > $$T_{analitic}(x, \ y,\ t\to \infty) = \dfrac{\sinh (\pi y) \sin (\pi x)}{\sinh \pi}$$